In [1]:
import sys
import os
# Add the parent directory to the Python path

# Change the current working directory to /app
os.chdir('/app')
from dotenv import load_dotenv
load_dotenv(dotenv_path='config/.env')
database_path = os.getenv('DATABASE_PATH')
from utils.db.fetch import fetch_entries
print(f'DATABASE_PATH: {database_path}')
market_name = 'crypto_binance'
timeframe='1m'
pair='BTC'

DATABASE_PATH: database/db/ohlcv_data.db


In [ ]:
from data.fetch.crypto_binance import fetch_symbol_list_binance
symbol_list = fetch_symbol_list_binance(type='spot', suffix='USDT')
symbol_list

In [5]:
len(symbol_list)

539

In [2]:
from finstore.finstore import Finstore
finstore = Finstore(market_name=market_name, timeframe=timeframe, enable_append=True)

Len items in 'database/finstore/market_name=crypto_binance/timeframe=1m': 257


In [3]:
df = finstore.stream.fetch_trade_data(symbol='ETHBTC')
df.tail(10)

,timestamp,open,high,low,close,volume,buy_volume
7528,1734794400000,0.03484,0.03484,0.03483,0.03483,10.5199,4.2260
7529,1734794460000,0.03483,0.03485,0.03483,0.03485,14.3768,11.5735
7530,1734794520000,0.03484,0.03484,0.03482,0.03482,40.8176,40.0745
7531,1734794580000,0.03483,0.03485,0.03482,0.03485,5.7402,3.3965
7532,1734794640000,0.03485,0.03487,0.03485,0.03485,10.8758,7.7157
7533,1734794700000,0.03485,0.03487,0.03485,0.03486,2.5237,2.4295
7534,1734794760000,0.03486,0.03489,0.03486,0.03488,15.6211,10.7903
7535,1734794820000,0.03486,0.03486,0.03485,0.03486,4.3466,1.3826
7536,1734794880000,0.03486,0.03486,0.03485,0.03486,7.1129,0.5967
7537,1734794940000,0.03486,0.03486,0.03484,0.03485,4.5132,3.2427


In [4]:
df = finstore.stream.fetch_trade_data(symbol='AAVEBTC')
df.tail(10)

,timestamp,open,high,low,close,volume,buy_volume
2602,1734361980000,0.003557,0.003564,0.003557,0.003562,2.341,0.924
2603,1734362040000,0.003563,0.003569,0.003563,0.003567,2.248,1.267
2604,1734362100000,0.003565,0.003565,0.003560,0.003562,2.639,1.532
2605,1734362160000,0.003564,0.003565,0.003549,0.003549,34.908,1.243
2606,1734362220000,0.003549,0.003559,0.003549,0.003559,6.594,5.657
2607,1734362280000,0.003564,0.003567,0.003560,0.003563,4.348,2.778
2608,1734362340000,0.003564,0.003569,0.003561,0.003567,20.970,18.345
2609,1734362400000,0.003568,0.003572,0.003565,0.003571,8.541,6.798
2610,1734362460000,0.003572,0.003572,0.003564,0.003564,5.399,1.499
2611,1734362520000,0.003564,0.003567,0.003563,0.003567,1.376,0.484


In [8]:
df.iloc[-1]['timestamp']

1734794940000.0

In [ ]:
from data.calculate.crypto_binance import calculate_technical_indicators
from data.fetch.crypto_binance import fetch_symbol_list_binance
market_name = 'crypto_binance'
symbol_list = fetch_symbol_list_binance(type='spot', suffix='BTC')
symbol_list = [symbol.replace('/', '') for symbol in symbol_list]
calculate_technical_indicators(market_name=market_name, symbol_list=symbol_list, timeframe=timeframe)

In [6]:
from finstore.finstore import Finstore
finstore = Finstore(market_name=market_name, timeframe=timeframe, enable_append=True)
symbol_list = finstore.read.get_symbol_list()
df = finstore.read.symbol_list(symbol_list=symbol_list, merged_dataframe=True)
df

Len items in 'database/finstore/market_name=crypto_binance/timeframe=1m': 257


{'BADGERBTC':           timestamp      open      high       low     close  volume  \
 0     1734205740000  0.000038  0.000038  0.000038  0.000038     0.0   
 1     1734205800000  0.000038  0.000038  0.000038  0.000038     0.0   
 2     1734205860000  0.000038  0.000038  0.000038  0.000038     0.0   
 3     1734205920000  0.000038  0.000038  0.000038  0.000038     0.0   
 4     1734205980000  0.000038  0.000038  0.000038  0.000038     0.0   
 ...             ...       ...       ...       ...       ...     ...   
 2608  1734362340000  0.000037  0.000037  0.000037  0.000037     0.0   
 2609  1734362400000  0.000037  0.000037  0.000037  0.000037     0.0   
 2610  1734362460000  0.000037  0.000037  0.000037  0.000037     0.0   
 2611  1734362520000  0.000037  0.000037  0.000037  0.000037     0.0   
 2612  1734362580000  0.000037  0.000037  0.000037  0.000037     0.0   
 
       buy_volume     market_name timeframe  average_volume_90   ema_100  \
 0            0.0  crypto_binance        1m  

In [4]:
for symbol in df:
    df[symbol]['average_volume_90'] = 0
    df[symbol].dropna(inplace=True)

In [12]:
df['ETHBTC'].tail(10)

,timestamp,open,high,low,close,volume,buy_volume,market_name,timeframe,average_volume_90,...,slope_r2_product_15,slope_r2_product_30,slope_r2_product_90,spike_30_0.85,supertrend_7_3,sustained_volume_spike_50_3_10,sustained_volume_spike_50_3_5,average_volume_50,volume_spike,slope_r2_volume_30
1970-01-01 00:00:02.603,1734362040000,0.03711,0.03713,0.03710,0.03712,20.8934,20.5998,crypto_binance,1m,1.331229,...,-2.009664e-04,-0.000290,-0.000113,0.0,0.037188,0.0,2.0,24.612160,0,-1.037897e-08
1970-01-01 00:00:02.604,1734362100000,0.03712,0.03713,0.03710,0.03713,82.6389,41.2609,crypto_binance,1m,1.364511,...,-1.122829e-04,-0.000280,-0.000116,0.0,0.037188,0.0,2.0,25.181140,0,1.385946e-08
1970-01-01 00:00:02.605,1734362160000,0.03712,0.03714,0.03709,0.03709,272.9482,179.6238,crypto_binance,1m,1.475033,...,-7.316328e-05,-0.000272,-0.000120,0.0,0.037188,0.0,2.0,28.506220,1,1.816485e-04
1970-01-01 00:00:02.606,1734362220000,0.03709,0.03712,0.03709,0.03711,25.6603,9.5379,crypto_binance,1m,1.476707,...,-2.984030e-05,-0.000264,-0.000124,0.0,0.037188,0.0,2.0,28.582202,0,9.657440e-05
1970-01-01 00:00:02.607,1734362280000,0.03712,0.03714,0.03710,0.03712,67.0094,28.0957,crypto_binance,1m,1.485111,...,-1.982637e-06,-0.000238,-0.000128,0.0,0.037188,0.0,2.0,29.133056,0,3.834518e-04
1970-01-01 00:00:02.608,1734362340000,0.03711,0.03714,0.03710,0.03713,89.7222,41.8447,crypto_binance,1m,1.517478,...,5.929335e-07,-0.000212,-0.000131,0.0,0.037188,0.0,2.0,29.956954,0,1.689584e-03
1970-01-01 00:00:02.609,1734362400000,0.03713,0.03715,0.03711,0.03714,14.0230,6.0358,crypto_binance,1m,1.515098,...,2.328053e-05,-0.000182,-0.000135,0.0,0.037188,0.0,2.0,29.709726,0,2.213263e-04
1970-01-01 00:00:02.610,1734362460000,0.03714,0.03714,0.03709,0.03709,137.9977,37.9903,crypto_binance,1m,1.571534,...,8.259456e-06,-0.000168,-0.000140,0.0,0.037188,0.0,2.0,30.445668,0,8.223579e-04
1970-01-01 00:00:02.611,1734362520000,0.03708,0.03708,0.03705,0.03707,36.5375,7.1622,crypto_binance,1m,1.585426,...,8.682160e-07,-0.000160,-0.000145,0.0,0.037183,0.0,2.0,30.428144,0,2.599532e-04
1970-01-01 00:00:02.612,1734362580000,0.03706,0.03707,0.03704,0.03706,32.2332,2.5563,crypto_binance,1m,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.040632,0,9.471952e-04


In [ ]:
import pandas as pd

# Iterate over each symbol in the DataFrame
for symbol in df:
    # Convert the timestamp to a readable datetime format
    df[symbol].index = pd.to_datetime(df[symbol].index, unit='ms')
    
    # Calculate the rolling average volume over the last 14 data points
    df[symbol]['average_volume_50'] = df[symbol]['buy_volume'].rolling(window=50).mean()
    
    # Create a new column to flag if the volume is greater than 5 times the average volume
    df[symbol]['volume_spike'] = (df[symbol]['buy_volume'] > 5 * df[symbol]['average_volume_50']).astype(int)

    df[symbol]['sustained_spike_rolling'] = df[symbol]['volume_spike'].rolling(3, min_periods=1).sum()

    # Step 4: Detect when a sustained spike starts
    df[symbol]['sustained_spike_temp'] = (df[symbol]['sustained_spike_rolling'] >= 3).astype(int)

    # Step 5: Propagate the flag until the pump ends
    # Forward-fill the sustained spike flag until volume no longer meets the threshold
    df[symbol]['sustained_spike'] = (
        df[symbol]['sustained_spike_temp'].cumsum().where(df[symbol]['volume_spike']).ffill().fillna(0).astype(int)
    )

# Collect all data points where a volume spike is detected
spike_data_points = []
for symbol in df:
    spikes = df[symbol][df[symbol]['sustained_spike'] >= 1]
    for index, row in spikes.iterrows():
        spike_data_points.append((symbol, index, row['buy_volume']))

# spike_data_points now contains tuples of (symbol, timestamp, volume) for each spike

In [ ]:
# Import the necessary function
from utils.calculation.slope_r2 import calculate_exponential_regression_volume

# Iterate over each symbol in the DataFrame
for symbol in df:
    # Calculate the exponential regression volume for each symbol
    df_calculated = calculate_exponential_regression_volume(df[symbol], window=30)
    
    # Add the calculated indicator as a new column to the original DataFrame
    df[symbol]['slope_r2_volume_30'] = df_calculated['indicator_value']

# Example: Display the last 10 rows of the updated DataFrame for 'ETHBTC'
df['XRPBTC'].tail(10)


In [13]:
# ... existing code ...

import pandas as pd

# Create an empty DataFrame to store all entries
all_entries = pd.DataFrame()

# Iterate over each symbol in the DataFrame
for symbol in df:
    # Add a column to identify the symbol
    df[symbol]['symbol'] = symbol
    df[symbol]['timestamp'] = pd.to_datetime(df[symbol]['timestamp'], unit='ms')
    
    # Append the DataFrame to the all_entries DataFrame
    all_entries = pd.concat([all_entries, df[symbol]], ignore_index=True)

# Sort the combined DataFrame by the slope_r2_volume_30 column in descending order
all_entries_sorted = all_entries.sort_values(by='slope_r2_volume_30', ascending=False)

# Select the top 10 entries
top_10_entries = all_entries_sorted.head(10)

# Display the top 10 entries
top_10_entries

,timestamp,open,high,low,close,volume,buy_volume,market_name,timeframe,average_volume_90,...,spike_30_0.85,supertrend_7_3,sustained_volume_spike_50_3_10,sustained_volume_spike_50_3_5,average_volume_50,volume_spike,slope_r2_volume_30,symbol,slope_r2_product_30,slope_r2_product_15
480568,2024-12-16 11:40:00,0.000034,0.000034,0.000034,0.000034,1226.310,743.000,crypto_binance,1m,0.020973,...,0.0,0.000034,0.0,0.0,392.23560,0,0.103007,MEBTC,-0.000308,-0.000136
480567,2024-12-16 11:39:00,0.000034,0.000034,0.000034,0.000034,9416.060,3410.210,crypto_binance,1m,0.020536,...,0.0,0.000034,0.0,0.0,377.37560,1,0.099475,MEBTC,-0.000302,-0.000341
654982,2024-12-16 00:43:00,0.003432,0.003437,0.003430,0.003435,13.536,13.319,crypto_binance,1m,0.039089,...,0.0,0.003449,0.0,3.0,4.04836,0,0.095869,AAVEBTC,-0.000623,-0.000987
655577,2024-12-16 10:38:00,0.003711,0.003730,0.003711,0.003718,83.303,71.970,crypto_binance,1m,0.048131,...,0.0,0.003682,0.0,4.0,5.75942,1,0.091471,AAVEBTC,0.000748,0.000796
655575,2024-12-16 10:36:00,0.003706,0.003706,0.003698,0.003701,96.034,3.929,crypto_binance,1m,0.044983,...,0.0,0.003667,0.0,4.0,4.15764,0,0.086231,AAVEBTC,0.000606,0.000903
480565,2024-12-16 11:37:00,0.000034,0.000034,0.000034,0.000034,9509.230,2887.980,crypto_binance,1m,0.016256,...,0.0,0.000034,0.0,0.0,288.50100,1,0.084856,MEBTC,-0.000282,-0.000554
655576,2024-12-16 10:37:00,0.003702,0.003715,0.003701,0.003714,11.130,8.777,crypto_binance,1m,0.044888,...,0.0,0.003672,0.0,4.0,4.32572,0,0.083871,AAVEBTC,0.000679,0.000851
430263,2024-12-16 00:42:00,0.000029,0.000029,0.000029,0.000029,2355.100,78.500,crypto_binance,1m,0.067583,...,0.0,0.000029,1.0,6.0,1794.44400,0,0.081465,WLDBTC,-0.000124,-0.000277
480566,2024-12-16 11:38:00,0.000034,0.000034,0.000034,0.000034,3200.480,1310.120,crypto_binance,1m,0.017058,...,0.0,0.000034,0.0,0.0,309.17140,0,0.080892,MEBTC,-0.000289,-0.000473
654980,2024-12-16 00:41:00,0.003430,0.003430,0.003424,0.003429,24.185,0.356,crypto_binance,1m,0.042306,...,0.0,0.003449,0.0,3.0,3.91264,0,0.078406,AAVEBTC,-0.000515,-0.001217


In [44]:
from strategy.private.crypto_sotm import CryptoSOTM
crypto_sotm = CryptoSOTM()
top_n_symbols = crypto_sotm.get_top_n_symbols(df, n=10, r2_period=30)
telegram_message = crypto_sotm.get_top_n_pairs_message(top_n_symbols, n=10)
telegram_message

'*Top 10 alt/BTC pairs:*\n• *KMDBTC*: `0.000494` (Score: 4/4)\n• *EIGENBTC*: `0.000407` (Score: 4/4)\n• *LDOBTC*: `0.000308` (Score: 4/4)\n• *SCRBTC*: `0.000283` (Score: 4/4)\n• *LITBTC*: `0.000239` (Score: 4/4)\n• *GLMRBTC*: `0.000237` (Score: 4/4)\n• *SUIBTC*: `0.000199` (Score: 4/4)\n• *BANANABTC*: `0.000194` (Score: 4/4)\n• *ZECBTC*: `0.000179` (Score: 4/4)\n• *DOTBTC*: `0.000168` (Score: 4/4)'

In [53]:
import pandas as pd

def detect_and_sort_largest_sustained_spikes(data_dict, lookback_period=50, min_sustained_periods=5):
    """
    Detects and sorts the largest sustained volume spikes for multiple symbols.

    Parameters:
    - data_dict: dict, where keys are symbols and values are OHLCV DataFrames
    - lookback_period: int, number of periods for calculating the average volume
    - min_sustained_periods: int, minimum consecutive periods with sustained volume spikes
    
    Returns:
    - sorted_spikes: DataFrame, sorted list of the largest sustained volume spikes across symbols
    """
    spikes_list = []
    
    for symbol, df in data_dict.items():
        if 'volume' not in df.columns:
            print(f"Volume column missing for {symbol}. Skipping.")
            continue
        
        # Ensure the data is sorted by time
        df = df.sort_index()

        # Calculate the rolling average volume
        df['avg_volume'] = df['buy_volume'].rolling(window=lookback_period, min_periods=lookback_period).mean()

        # Calculate volume multiplier
        df['volume_multiplier'] = df['buy_volume'] / df['avg_volume']

        # Identify sustained spikes for the required number of periods
        df['sustained_spike'] = df['volume_multiplier'].rolling(window=min_sustained_periods).min()  # Min over 3 periods
        df['is_sustained'] = df['sustained_spike'] >= 1  # Keep sustained spikes

        # Filter rows where sustained spikes occur
        sustained_spikes = df[df['is_sustained'] & df['sustained_spike'].notna()].copy()

        # Add symbol and spike information to the list
        for _, row in sustained_spikes.iterrows():
            # Calculate the index of the row 10 candles after the current spike
            future_index = df.index.get_loc(row.name) + 20
            
            # Ensure the future index is within the DataFrame bounds
            if future_index < len(df):
                future_close = df.iloc[future_index]['close']
                percent_change = ((future_close - row['close']) / row['close']) * 100
            else:
                percent_change = None  # or handle as you see fit, e.g., 0 or np.nan

            spikes_list.append({
                'symbol': symbol,
                'timestamp': pd.to_datetime(row['timestamp'], unit='ms'),  # Assuming the index is timestamp
                'sustained_multiplier': row['sustained_spike'],
                'volume': row['buy_volume'],
                'usd_volume': row['buy_volume'] * row['close'] * 100000,
                'avg_volume': row['avg_volume'],
                'percent_change_10_candles': percent_change
            })
    
    # Create a DataFrame of all spikes
    spikes_df = pd.DataFrame(spikes_list)

    # Sort by the sustained multiplier in descending order
    sorted_spikes = spikes_df.sort_values(by='sustained_multiplier', ascending=False).reset_index(drop=True)
    
    return sorted_spikes

# Example Usage:
# Assuming `ohlcv_dict` is your dictionary of OHLCV data with symbols as keys
sorted_sustained_spikes = detect_and_sort_largest_sustained_spikes(df)



In [54]:
sorted_sustained_spikes.sort_values(by='percent_change_10_candles', ascending=False).head(20)

,symbol,timestamp,sustained_multiplier,volume,usd_volume,avg_volume,percent_change_10_candles
247,VIBBTC,2024-12-15 04:35:00,2.269651,45735.00,4482.0300,4431.100000,8.163265
1082,CTXCBTC,2024-12-15 18:07:00,1.065202,1408.00,782.8480,1072.200000,6.654676
1135,VIBBTC,2024-12-15 04:46:00,1.037023,27458.00,2828.1740,9747.340000,5.825243
248,VIBBTC,2024-12-15 04:34:00,2.269651,7981.00,790.1190,3516.400000,5.050505
132,SLFBTC,2024-12-15 11:22:00,3.059205,4247.80,2523.1932,695.720000,4.882155
237,ENSBTC,2024-12-15 23:20:00,2.308083,4.54,194.5844,1.967000,4.549697
320,MOVEBTC,2024-12-16 00:10:00,2.001442,155.50,95.4770,77.694000,4.234528
904,ADXBTC,2024-12-14 20:51:00,1.180743,64353.00,18404.9580,25549.980000,3.846154
1033,ADXBTC,2024-12-14 20:52:00,1.086641,27990.00,8033.1300,25758.280000,3.832753
965,VIBBTC,2024-12-15 07:20:00,1.127422,10657.00,1150.9560,9452.540000,3.703704


In [32]:
sorted_sustained_spikes[sorted_sustained_spikes['symbol'] == 'FTMBTC'].head(20)


,symbol,timestamp,sustained_multiplier,volume,avg_volume
207,FTMBTC,2024-12-15 14:25:00,9.347510,39235.0,3342.24
216,FTMBTC,2024-12-14 21:42:00,9.187381,8775.0,659.86
361,FTMBTC,2024-12-15 17:33:00,7.264913,52753.0,7261.34
481,FTMBTC,2024-12-14 23:47:00,6.348121,2135.0,336.32
499,FTMBTC,2024-12-16 04:55:00,6.276399,9005.0,1434.74
517,FTMBTC,2024-12-15 01:55:00,6.149749,26991.0,3099.00
1058,FTMBTC,2024-12-15 18:35:00,4.234593,26378.0,4305.68
1331,FTMBTC,2024-12-15 01:56:00,3.648585,12197.0,3342.94
1407,FTMBTC,2024-12-14 21:43:00,3.524440,2502.0,709.90
1495,FTMBTC,2024-12-15 08:11:00,3.391660,5846.0,1723.64
